In [108]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.base import BaseEstimator, TransformerMixin

class TargetEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, categories='auto', k=1, f=1, 
                 noise_level=0, random_state=None):
        if type(categories)==str and categories!='auto':
            self.categories = [categories]
        else:
            self.categories = categories
        self.k = k
        self.f = f
        self.noise_level = noise_level
        self.encodings = dict()
        self.prior = None
        self.random_state = random_state
        
    def add_noise(self, series, noise_level):
        return series * (1 + noise_level *   
                         np.random.randn(len(series)))
        
    def fit(self, X, y=None):
        if type(self.categories)=='auto':
            self.categories = np.where(X.dtypes == type(object()))[0]
        
        temp = X.loc[:, self.categories].copy()
        temp['target'] = y
        self.prior = np.mean(y)
        for variable in self.categories:
            avg = (temp.groupby(by=variable)['target']
                       .agg(['mean', 'count']))
            # Compute smoothing 
            smoothing = (1 / (1 + np.exp(-(avg['count'] - self.k) /                 
                         self.f)))
            # The bigger the count the less full_avg is accounted
            self.encodings[variable] = dict(self.prior * (1 -  
                             smoothing) + avg['mean'] * smoothing)
            
        return self
    
    def transform(self, X, y=None):
        Xt = X.copy()
        for variable in self.categories:
            Xt[variable].replace(self.encodings[variable], 
                                 inplace=True)
            unknown_value = {value:self.prior for value in 
                             X[variable].unique() 
                             if value not in 
                             self.encodings[variable].keys()}
            if len(unknown_value) > 0:
                Xt[variable].replace(unknown_value, inplace=True)
            Xt[variable] = Xt[variable].astype(float)
            if self.noise_level > 0:
                if self.random_state is not None:
                    np.random.seed(self.random_state)
                Xt[variable] = self.add_noise(Xt[variable], 
                                              self.noise_level)
                
        Xt = Xt.fillna(self.prior)
        return Xt
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

In [126]:
class TargetEncoderCV():
    """Cross-fold target encoder.
    """
    
    def __init__(self, cols, k=1, f=1, cv=None, n_splits=3, shuffle=True):
        """Cross-fold target encoding for categorical features.
        
        Parameters
        ----------
        te: TargetEncoder object
        n_splits : int
            Number of cross-fold splits. Default = 3.
        shuffle : bool
            Whether to shuffle the data when splitting into folds.

        """
        self.cols = cols
        self.k = k
        self.f = f
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.cv = cv

        

    def fit(self, X, y):
        """Fit cross-fold target encoder to X and y
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values.
            
        Returns
        -------
        self : encoder
            Returns self.
        """
        self._target_encoder = TargetEncoder(self.cols, self.k, self.f)
        self._target_encoder.fit(X, y)
        return self

    
    def transform(self, X, y=None):
        """Perform the target encoding transformation.

        Uses cross-fold target encoding for the training fold,
        and uses normal target encoding for the test fold.

        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """

        # Use target encoding from fit() if this is test data
        if y is None:
            return self._target_encoder.transform(X)

        # Compute means for each fold
        self._train_ix = []
        self._test_ix = []
        self._fit_tes = []
        
        if self.cv is None:
            cv = KFold(n_splits=self.n_splits, shuffle=self.shuffle)
        
        for train_ix, test_ix in cv.split(X, y):
            self._train_ix.append(train_ix)
            self._test_ix.append(test_ix)
            te = TargetEncoder(self.cols, self.k, self.f)
            if isinstance(X, pd.DataFrame):
                self._fit_tes.append(te.fit(X.iloc[train_ix,:],
                                            y.iloc[train_ix]))
            elif isinstance(X, np.ndarray):
                self._fit_tes.append(te.fit(X[train_ix,:], y[train_ix]))
            else:
                raise TypeError('X must be DataFrame or ndarray')

        # Apply means across folds
        Xo = X.copy()
        for ix in range(len(self._test_ix)):
            test_ix = self._test_ix[ix]
            if isinstance(X, pd.DataFrame):
                Xo.iloc[test_ix,:] = self._fit_tes[ix].transform(X.iloc[test_ix,:])
            elif isinstance(X, np.ndarray):
                Xo[test_ix,:] = self._fit_tes[ix].transform(X[test_ix,:])
            else:
                raise TypeError('X must be DataFrame or ndarray')
        return Xo

            
    def fit_transform(self, X, y=None):
        """Fit and transform the data via target encoding.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values (required!).

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        return self.fit(X, y).transform(X, y)

In [127]:
data = {
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Houston'],
    'City2': ['NY', 'TK', 'DH', 'NY', 'tokyo'],
    'Population': [8419000, 3980000, 2716000, 2328000, 1690000],
    'Area (sq mi)': [302.6, 468.7, 227.3, 637.4, 517.6],
    'Established': [1624, 1781, 1837, 1837, 1867]
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)

In [134]:
# te = TargetEncode(, 1, 1)


enc = TargetEncoder(['City', 'City2'])

In [135]:
# enc.fit(df, df['Area (sq mi)'])

In [136]:
data = {
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'tokyo'],
    'City2': ['NY', 'TK', 'DH', 'NY', 'bd'],
    'Population': [8419000, 3980000, 2716000, 2328000, 1690000],
    'Area (sq mi)': [1, 0, 1, 0, 0],
    'Established': [1624, 1781, 1837, 1837, 1867]
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)

In [137]:
# enc.fit_transform(df, df["Established"])

In [138]:
enc.fit_transform(df, df['Area (sq mi)'])

,City,City2,Population,Area (sq mi),Established
0,0.7,0.473106,8419000,1,1624
1,0.2,0.200000,3980000,0,1781
2,0.7,0.700000,2716000,1,1837
3,0.2,0.473106,2328000,0,1837
4,0.2,0.200000,1690000,0,1867


In [139]:
enc.transform(df, df['Area (sq mi)'])

,City,City2,Population,Area (sq mi),Established
0,0.7,0.473106,8419000,1,1624
1,0.2,0.200000,3980000,0,1781
2,0.7,0.700000,2716000,1,1837
3,0.2,0.473106,2328000,0,1837
4,0.2,0.200000,1690000,0,1867


In [140]:
df

,City,City2,Population,Area (sq mi),Established
0,New York,NY,8419000,1,1624
1,Los Angeles,TK,3980000,0,1781
2,Chicago,DH,2716000,1,1837
3,Houston,NY,2328000,0,1837
4,tokyo,bd,1690000,0,1867


In [125]:
(1 / (1 + np.exp(-(2 - 1) /                 
                         1)))

0.7310585786300049

In [119]:
2/5

0.4